# Interpretability Case Study: Loan Data

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import statsmodels.api as sm
import shap

import Datasets as DS
from ruxg import RUGClassifier
from auxFunctions import get_leaves

import warnings
warnings.filterwarnings("ignore")

## Loading the data

The raw data and details for data cleaning are provided here: https://towardsdatascience.com/how-to-develop-a-credit-risk-model-and-scorecard-91335fc01f03 

In [ ]:
loan_data = DS.loan('../datasets/')
print(loan_data.shape)
loan_data.head()

In [ ]:
# Split the data with stratification
X = loan_data.drop('good_bad', axis = 1)
y = loan_data[['good_bad']]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state = 42, stratify = y)

X_train, X_test = X_train.copy(), X_test.copy()

## Table 1

We start with the objective of obtaining a high F1 score and, at the same time, providing interpretation for the results (if possible). 

### Logistic Regression (LR)

In [ ]:
X_train_aug = sm.add_constant(X_train)
X_test_aug =  sm.add_constant(X_test)

lr = sm.Logit(Y_train, X_train_aug).fit()

threshold = 0.5
Y_test['loan_pred'] = np.array(lr.predict(X_test_aug) > threshold , dtype=int)

In [ ]:
# Confusion Matrix
cm = pd.crosstab(Y_test['good_bad'], Y_test['loan_pred'])
print ("Confusion matrix : \n", cm)

# Accuracies
print('\nAccuracy  = %.4f' % accuracy_score(Y_test['good_bad'], Y_test['loan_pred']))
print('F1 score  = %.4f' % f1_score(Y_test['good_bad'], Y_test['loan_pred']))

### Decision Tree (DT)

In [ ]:
tc = DecisionTreeClassifier(criterion='gini', max_depth = 2, random_state=1)
tc = tc.fit(X_train, Y_train)

rule_length = get_leaves(tc, X_test)
Y_test['loan_pred'] = tc.predict(X_test)

In [ ]:
# Confusion Matrix
cm = pd.crosstab(Y_test['good_bad'], Y_test['loan_pred'])
print ("Confusion matrix : \n", cm)

# Accuracies
print('\nAccuracy  = %.4f' % accuracy_score(Y_test['good_bad'], Y_test['loan_pred']))
print('F1 score  = %.4f' % f1_score(Y_test['good_bad'], Y_test['loan_pred']))
print('Number of rules = %.0f' % tc.get_n_leaves())
print('Average rule length = %.2f' % np.array(rule_length).mean())

### Random Forest (RF)

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=7, random_state=1)
rfc.fit(np.array(X_train), np.array(Y_train).flatten())

Y_test['loan_pred'] = rfc.predict(X_test)

In [ ]:
# Confusion matrix
cm = pd.crosstab(Y_test['good_bad'], Y_test['loan_pred'])
print ("Confusion matrix : \n", cm)

n_leaves = 0
for dtc in rfc.estimators_:
    n_leaves += dtc.get_n_leaves()

print('\nAccuracy  = %.4f' % accuracy_score(Y_test['good_bad'], Y_test['loan_pred']))
print('F1 score  = %.4f' % f1_score(Y_test['good_bad'], Y_test['loan_pred']))
print('Number of rules = %.0f' % n_leaves)

### AdaBoost (ADA)

In [ ]:
ada = AdaBoostClassifier(n_estimators = 30, random_state=1)
ada.fit(X_train, Y_train)
Y_test['loan_pred'] = ada.predict(X_test)

In [ ]:
# Confusion matrix
cm = pd.crosstab(Y_test['good_bad'], Y_test['loan_pred'])
print ("Confusion matrix : \n", cm)

# n_leaves = sum(tree.tree_.n_leaves for tree in gbc.estimators_.reshape(-1))

print('\nAccuracy  = %.4f' % accuracy_score(Y_test['good_bad'], Y_test['loan_pred']))
print('F1 score  = %.4f' % f1_score(Y_test['good_bad'], Y_test['loan_pred']))
print('Number of rules = %.0f' % (2*ada.n_estimators))

### Gradient Boosting (GB)

In [ ]:
gbc = GradientBoostingClassifier(max_depth = 2, n_estimators = 30, random_state=1)
gbc.fit(X_train, Y_train)
Y_test['loan_pred'] = gbc.predict(X_test)

In [ ]:
# Confusion matrix
cm = pd.crosstab(Y_test['good_bad'], Y_test['loan_pred'])
print ("Confusion matrix : \n", cm)

n_leaves = sum(tree.tree_.n_leaves for tree in gbc.estimators_.reshape(-1))

print('\nAccuracy  = %.4f' % accuracy_score(Y_test['good_bad'], Y_test['loan_pred']))
print('F1 score  = %.4f' % f1_score(Y_test['good_bad'], Y_test['loan_pred']))
print('Number of rules = %.0f' % n_leaves)

### RUG

To obtain a simple and interpretable model with a better F1 score, we tune the RUG model. We choose the maximum depth as two, and maximum number of rule generations as three.

In [ ]:
Xnp = np.array(X_train)
Ynp = np.array(Y_train).flatten()

RUG = RUGClassifier(max_depth=2, max_RMP_calls=3,
                    rule_length_cost=True, solver='gurobi', random_state=1)
RUG_fit = RUG.fit(Xnp, Ynp)

Y_test['loan_pred'] = RUG.predict(np.array(X_test))

In [ ]:
# Confusion matrix
cm = pd.crosstab(Y_test['good_bad'], Y_test['loan_pred'])
print ("Confusion matrix : \n", cm)

print('\nAccuracy  = %.4f' % accuracy_score(Y_test['good_bad'], Y_test['loan_pred']))
print('F1 score  = %.4f' % f1_score(Y_test['good_bad'], Y_test['loan_pred']))
print('Number of rules = %.0f' % RUG.get_num_of_rules())
print('Average rule length = %.2f' % RUG.get_avg_rule_length())

dict1, dict2 = RUG.get_instance_to_rule_dicts(X_test.index, X_test)
print('Average number of rules used per sample: %.2f' % np.mean(list(dict2.values())))

## Figure 2

### Print rules generated by RUG

In [ ]:
RUG.print_rules(feature_names=X_train.columns)

### Compare with SHAP applied to GB

Now, let us explain the predictions of GB model with SHAP and then compare it against RUG's output.

In [ ]:
shap.initjs()
explainer = shap.TreeExplainer(gbc, X_test)
shap_values = explainer(X_test)

For instance 6

In [ ]:
i = 6
id = X_test.index[i]

# rules covering that instance
RUG.print_rules_for_instances([id], dict1, colnames=X_test.columns)

# shap
shap.plots.waterfall(shap_values[i])

For instance 18

In [ ]:
i = 18
id = X_test.index[i]

# rules covering that instance
RUG.print_rules_for_instances([id], dict1, colnames=X_test.columns)

# shap
shap.plots.waterfall(shap_values[i])

## Table 2

### Logistic Regression

see above

### Decision Tree (DT)

In [ ]:
pgrid = {'max_depth': [3,5,7,9,11,13,15]}

tc_estimator = DecisionTreeClassifier(criterion='gini', random_state=1)
gcv = GridSearchCV(estimator=tc_estimator, param_grid=pgrid, n_jobs=1, cv=5, verbose=0, refit=True)
gcv_fit = gcv.fit(X_train, Y_train)
tc = gcv_fit.best_estimator_

Y_test['loan_pred'] = tcc.predict(X_test)

In [ ]:
gcv_fit.best_estimator_

In [ ]:
# Confusion Matrix
cm = pd.crosstab(Y_test['good_bad'], Y_test['loan_pred'])
print ("Confusion matrix : \n", cm)

# Accuracies
print('\nAccuracy  = %.4f' % accuracy_score(Y_test['good_bad'], Y_test['loan_pred']))
print('F1 score  = %.4f' % f1_score(Y_test['good_bad'], Y_test['loan_pred']))
print('Number of rules = %.0f' % tc.get_n_leaves())
print('Average rule length = %.2f' % np.array(rule_length).mean())

### Random Forest (RF)

In [ ]:
pgrid = {'max_depth': [3,5,7,9,11,13,15], 
         'n_estimators':[100, 150, 200,250,300]}

rf_estimator = RandomForestClassifier(criterion='gini', random_state=1)
gcv = GridSearchCV(estimator=rf_estimator, param_grid=pgrid, n_jobs=1, cv=5, verbose=0, refit=True)
gcv_fit = gcv.fit(X_train, Y_train)
rfc = gcv_fit.best_estimator_

Y_test['loan_pred'] = rfc.predict(X_test)

In [ ]:
gcv_fit.best_estimator_

In [ ]:
# Confusion matrix
cm = pd.crosstab(Y_test['good_bad'], Y_test['loan_pred'])
print ("Confusion matrix : \n", cm)

n_leaves = 0
for dtc in rfc.estimators_:
    n_leaves += dtc.get_n_leaves()

print('\nAccuracy  = %.4f' % accuracy_score(Y_test['good_bad'], Y_test['loan_pred']))
print('F1 score  = %.4f' % f1_score(Y_test['good_bad'], Y_test['loan_pred']))
print('Number of rules = %.0f' % n_leaves)

### ADA

In [ ]:
pgrid = {'n_estimators':[100,150,200,250,30]}

ada_estimator = AdaBoostClassifier(random_state=1)    
gcv = GridSearchCV(estimator=ada_estimator, param_grid=pgrid, n_jobs=1, cv=5, verbose=0, refit=True)
gcv_fit = gcv.fit(X_train, Y_train)
ada = gcv_fit.best_estimator_

Y_test['loan_pred'] = ada.predict(X_test)

In [ ]:
gcv_fit.best_estimator_

In [ ]:
# Confusion matrix
cm = pd.crosstab(Y_test['good_bad'], Y_test['loan_pred'])
print ("Confusion matrix : \n", cm)

# n_leaves = sum(tree.tree_.n_leaves for tree in gbc.estimators_.reshape(-1))

print('\nAccuracy  = %.4f' % accuracy_score(Y_test['good_bad'], Y_test['loan_pred']))
print('F1 score  = %.4f' % f1_score(Y_test['good_bad'], Y_test['loan_pred']))
print('Number of rules = %.0f' % (2*ada.n_estimators))

### GB

In [ ]:
pgrid = {'max_depth':[3,5,7,9,11,15],
         'n_estimators':[100,150,200,250,30]}

gb_estimator = GradientBoostingClassifier(random_state=1)
gcv = GridSearchCV(estimator=gb_estimator, param_grid=pgrid, n_jobs=1, cv=5, verbose=0, refit=True)
gcv_fit = gcv.fit(X_train, Y_train)
gbc = gcv_fit.best_estimator_

Y_test['loan_pred'] = gbc.predict(X_test)

In [ ]:
gcv_fit.best_estimator_

In [ ]:
# Confusion matrix
cm = pd.crosstab(Y_test['good_bad'], Y_test['loan_pred'])
print ("Confusion matrix : \n", cm)

n_leaves = sum(tree.tree_.n_leaves for tree in gbc.estimators_.reshape(-1))

print('\nAccuracy  = %.4f' % accuracy_score(Y_test['good_bad'], Y_test['loan_pred']))
print('F1 score  = %.4f' % f1_score(Y_test['good_bad'], Y_test['loan_pred']))
print('Number of rules = %.0f' % n_leaves)

### RUG

In [ ]:
Xnp = np.array(X_train)
Ynp = np.array(Y_train).flatten()

RUG = RUGClassifier(max_depth=2, rule_length_cost=True, 
                    solver='gurobi', random_state=21)
RUG_fit = RUG.fit(Xnp, Ynp)

Y_predict = RUG.predict(np.array(X_test))
Y_test['loan_pred'] = Y_predict

In [ ]:
# Confusion matrix
cm = pd.crosstab(Y_test['good_bad'], Y_test['loan_pred'])
print ("Confusion matrix : \n", cm)

print('\nAccuracy  = %.4f' % accuracy_score(Y_test['good_bad'], Y_test['loan_pred']))
print('F1 score  = %.4f' % f1_score(Y_test['good_bad'], Y_test['loan_pred']))
print('Number of rules = %.0f' % RUG.get_num_of_rules())
print('Average rule length = %.2f' % RUG.get_avg_rule_length())

dict1, dict2 = RUG.get_instance_to_rule_dicts(X_test.index, X_test)
print('Average number of rules used per sample: %.2f' % np.mean(list(dict2.values())))